In [5]:
%load_ext dotenv
%dotenv

import os

%cd {os.getenv("PROJECT_PATH") or ".."}

%load_ext autoreload
%autoreload 1

from IPython.display import display


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
cannot find .env file
/data03/home/gary.wei/projects/evagpt
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
from pathlib import Path
from absl import logging
from tqdm.notebook import tqdm
from timeit import default_timer as timer
import pickle

logging.set_verbosity(logging.INFO)


In [7]:
from pandarallel import pandarallel

pandarallel.initialize(
    nb_workers=os.cpu_count(),
    progress_bar=True,
    verbose=0
)


In [8]:
def show_df(df: pd.DataFrame):
    display(df.head())
    print(df.shape)


In [9]:
from datasets import load_dataset_builder

In [10]:
ds_builder = load_dataset_builder("wikitext", "wikitext-2-raw-v1")

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

In [12]:
ds_builder.info

DatasetInfo(description='', citation='', homepage='', license='', features={'text': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, builder_name='parquet', dataset_name='wikitext', config_name='wikitext-2-raw-v1', version=0.0.0, splits={'test': SplitInfo(name='test', num_bytes=1305088, num_examples=4358, shard_lengths=None, dataset_name=None), 'train': SplitInfo(name='train', num_bytes=11061717, num_examples=36718, shard_lengths=None, dataset_name=None), 'validation': SplitInfo(name='validation', num_bytes=1159288, num_examples=3760, shard_lengths=None, dataset_name=None)}, download_checksums=None, download_size=7747362, post_processing_size=None, dataset_size=13526093, size_in_bytes=None)

In [14]:
from datasets import load_dataset

dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes", split="train")

README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [15]:
from datasets import get_dataset_split_names

get_dataset_split_names("cornell-movie-review-data/rotten_tomatoes")

['train', 'validation', 'test']

In [16]:
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 8530
})

In [17]:
dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")

In [18]:
from datasets import get_dataset_config_names

configs = get_dataset_config_names("PolyAI/minds14")
configs

README.md:   0%|          | 0.00/5.28k [00:00<?, ?B/s]

minds14.py:   0%|          | 0.00/5.83k [00:00<?, ?B/s]

['cs-CZ',
 'de-DE',
 'en-AU',
 'en-GB',
 'en-US',
 'es-ES',
 'fr-FR',
 'it-IT',
 'ko-KR',
 'nl-NL',
 'pl-PL',
 'pt-PT',
 'ru-RU',
 'zh-CN',
 'all']

In [20]:
c4 = load_dataset("allenai/c4", "en", split="train", trust_remote_code=True)

/data03/home/gary.wei/.cache/huggingface/modules/datasets_modules/datasets/c4/584d57ebe81c209b6c7f31727066d2c4b4bba37cb7092cdd83083d5ec11207db/c4.py:53: FutureWarning: Dataset 'c4' is deprecated and will be deleted. Use 'allenai/c4' instead.
  warnings.warn(


en/c4-train.00000-of-01024.json.gz:   0%|          | 0.00/319M [00:00<?, ?B/s]

en/c4-train.00001-of-01024.json.gz:   0%|          | 0.00/318M [00:00<?, ?B/s]

en/c4-train.00002-of-01024.json.gz:   0%|          | 0.00/320M [00:00<?, ?B/s]

en/c4-train.00003-of-01024.json.gz:   0%|          | 0.00/319M [00:00<?, ?B/s]

...

en/c4-train.00005-of-01024.json.gz:   0%|          | 0.00/318M [00:00<?, ?B/s]

en/c4-train.00006-of-01024.json.gz:   0%|          | 0.00/318M [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.


en/c4-train.00007-of-01024.json.gz:   0%|          | 0.00/318M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [21]:
dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes", split="train")

In [22]:
dataset[0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [23]:
dataset[-1]

{'text': 'things really get weird , though not particularly scary : the movie is all portent and no content .',
 'label': 0}

In [24]:
dataset["text"]

['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
 'effective but too-tepid biopic',
 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .',
 "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .",
 'the film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .',
 'offers that rare combination of entertainment and education .',
 'perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .',
 "steers turns i

In [25]:
dataset[0]["text"]

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'

In [26]:
dataset["text"][0]

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'

In [27]:
iterable_dataset = load_dataset(
    "ethz/food101", split="train", streaming=True, trust_remote_code=True
)

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

In [30]:
next(iter(iterable_dataset))

{'image': <PIL.Image.Image image mode=RGB size=384x512>, 'label': 6}

In [31]:
from datasets import load_dataset

In [33]:
dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes", split="train")
iterable_dataset = dataset.to_iterable_dataset()